# STWAVE test

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import fsspec

### Read the data using the cloud-friendly zarr data format

In [ ]:
ds = xr.open_zarr(fsspec.get_mapper('s3://coastalcoupling/usace/stwave_October2015.zarr',
            anon=False, requester_pays=True))

### Start a dask cluster to crunch the data

In [ ]:
from dask_gateway import Gateway

from dask.distributed import Client

gateway = Gateway()

cluster = gateway.new_cluster(environment='pangeo', profile='Small Worker')

In [ ]:
cluster.scale(14);

In [ ]:
client = Client(cluster)

### Visualize using HoloViz.org tools

Take the maximum over the time dimension and persist the data on the workers to use later.  This is the computationally intensive step.

In [ ]:
%%time 
max_var = ds['waveHs'].max(dim='time').compute()

In [ ]:
max_var.to_netcdf('/home/jovyan/shared/users/rsignell/testing/new.nc')

In [ ]:
cluster.shutdown()